# Define a PWFA driver bunch and add it to a PIConGPU simulation

## Intro

This notebook allows you to define a PWFA driver bunch using numpy.
Each step will be explained on the way.
In the end, you will add your particles (of the driver bunch) to an empty **openPMD-api** checkpoint in **HDF5** format.
Since rewriting the entire code would have taken too long (before you start Nico), you have to stick to **HDF5** for now. 

## load modules

In [ ]:
# standard modules
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy import constants
import h5py

import sys

In [ ]:
# own modules
sys.path.append("./modules4picongpu/")

from bunchInit_openPMD_bp import vec3D
from bunchInit_openPMD_bp import addParticles2Checkpoint

## Generate a electron bunch at a known location
For our L|PWFA setups, we usually assume that the driver bunch of the 2nd stage/PWFA stage - which originate as witness bunch from the first stage/LWFA stage - is Gaussian/uncorrelated when exiting the LWFA stage and thus easy to define.

### define bunch parameters

In [ ]:
# set these values

Q = 400e-12 # define total charge of driver in [C]
radius_LWFA_exit_rms = 1.0e-6 * 10.0 # rms radius of bunch in [m]
tau_FWHM = 20.0e-15 # FWHM duration of bunch in [s]
E_kin_mean = 250.0e6 # mean energy in [eV]
E_kin_FWHM = 10.0e6 # energy spread in [eV]
theta_sigma = 1.6e-3 # standard deviation of divergence in [rad]
mean_weight = 50000 # define a constant weight for all macro-particles

In [ ]:
# compute relevant quantities

# number of macro particles:
N = Q / constants.elementary_charge / mean_weight

# convert rms to std of radius
sigma_LWFA_exit = radius_LWFA_exit_rms / np.sqrt(2)

# monte carlo: transversal position
x = np.random.normal(scale=sigma_LWFA_exit, size=int(N))
z = np.random.normal(scale=sigma_LWFA_exit, size=int(N))

# constant conversion_factor, equals 2*sqrt(2*ln(2))
const_FWHM_to_sigma = 2.35482004503

# convert FWHM to std of duration
sigma_y = tau_FWHM * constants.c / const_FWHM_to_sigma

# monte-carlo longitudinal distribution
y = np.random.normal(scale=sigma_y, size=int(N))

# monte-carlo energy distribution
E_kin = np.random.normal(loc=E_kin_mean, scale=E_kin_FWHM/const_FWHM_to_sigma, size=int(N))

# monte-carlo azimutal angle
theta = np.random.normal(loc=0.0, scale=theta_sigma*np.sqrt(2), size=int(N))
# monte-carlo polar angle (uniform distribution)
phi = np.random.uniform(low=-np.pi, high=+np.pi, size=int(N))

# convert kinetic energy to absolute momentum
convert_Ekin_to_momentum = (E_kin*constants.elementary_charge)/constants.c * mean_weight
# distribute to direction
px = np.sin(phi) * np.sin(theta) * convert_Ekin_to_momentum
py = 1.0 * np.cos(theta) * convert_Ekin_to_momentum
pz = np.cos(phi) * np.sin(theta) * convert_Ekin_to_momentum


## Getting particles into the PIC code

In this section, we will prepare the particle distribution to go into the empty checkpoint of PIConGPU.

### Parameters from the actual simulation
Get the resolution of the PIConGPU simulation. These parameters can be found in `include/picongpu/param/grid.param` and the `*.cfg` file you are using.

In [ ]:
# set parameters
# TODO: should be extracted from the checkpoint directly in the future
delta_t = 1.706e-16 / 1.28631 # [s]
delta_x = 0.5 * 0.1772e-6 # [m]
delta_y = delta_x # [m] 
delta_z = delta_x # [m]

In [ ]:
# set distribution to GPUs
cells = (1024, 2048, 1024)
GPUs = (2, 8, 2)
print("total number of GPUs:", np.product(GPUs))

## place particles inside simulation box
So far, we used a simple coordinate system. Now, we have to decide were to place the center (mean position) of the bunch inside our simulation box. 

In [ ]:
# define center position
center_pos_x = delta_x * cells[0]/2.
center_pos_y = delta_y * cells[1] * 2.4/4.
center_pos_z = delta_z * cells[2]/2.

In [ ]:
# just a variable rename
N_particles = int(N)

# make weighting an array (all macro-particle have the same weighting)
weighting = np.ones(N_particles) * mean_weight

# shift particles to new center position
x_PIConGPU = x + center_pos_x
y_PIConGPU = y - np.mean(y) + center_pos_y 
z_PIConGPU = z + center_pos_z

## convert data and write to checkpoint

In [ ]:
# convert data to 3d vector object
pos = vec3D(x_PIConGPU,y_PIConGPU,z_PIConGPU)
mom = vec3D(px/mean_weight, py/mean_weight, pz/mean_weight)

In [ ]:
# assigne to (existing) checkpoint file (needs to be hdf5 file)
checkPoint_b = addParticles2Checkpoint("/bigdata/hplsim/production/wrobel45/PWFA-bunch-slice/001_empty_slice/simOutput/checkpoints/checkpoint_%T.bp", 
                                       "/bigdata/hplsim/production/wrobel45/PWFA-bunch-energy/103_particles_energy/simOutput/checkpoint_%06T.bp",
                                       speciesName="b")

# this will throw an error if particles are already in the checkpoint - make sure you use an empty checkpoint

In [ ]:
# write data to file
checkPoint_b.addParticles(pos, mom, weighting)
checkPoint_b.writeParticles()

# delete data we wrote
del(checkPoint_b)